In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")
from helper_functions import importTensorflow
# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy('mixed_float16')
importTensorflow(memory=4090, precision=True)

[INFO] 'helper_functions.py' already exists, skipping download.
2.13.0
1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6
<Policy "mixed_float16">


# TRANSFER LEARNING PART 1 : FEATURE EXTRACTION
Transfer learning is leavraging a working neural architecture and use it for learning patterns for our problem

### Download and become one with the data

In [2]:
import os
if "10_food_classes_10_percent" not in os.listdir():
    !wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
    !unzip 10_food_classes_10_percent.zip
for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

--2023-12-31 23:47:43--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4007:822::201b, 2404:6800:4007:823::201b, 2404:6800:4007:824::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4007:822::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  16.5MB/s    in 11s     

2023-12-31 23:47:54 (15.2 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]

Archive:  10_food_classes_10_percent.zip
   creating: 10_food_classes_10_percent/
  inflating: __MACOSX/._10_food_classes_10_percent  
   creating: 10_food_classes_10_percent/test/
  inflating: __MACOSX/10_food_classes_10_percent/._test  
   creating: 10_food_classes_10_percent/train/
  inflating: __MACOSX/10_food_classes_10

In [3]:
## Prepare the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### Setting up callbacks:
things to run while the models are running, these are extra functionality added to our model during or after training

In [4]:
# Create a TensorBoard Callback (functionalized because we need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name+"/"+experiment_name+"/"+datetime.datetime.now().strftime("%Y%m%D-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
    print(f"Saving TensorBoard logfiles to: {log_dir}")
    return tensorboard_callback

## Using Tensorflow-Hub

In [16]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Compare two models
resnet = "https://www.kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-feature-vector/versions/2"
efficientnet = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

def create_model(model_url,num_of_class=10):
    model = tf.keras.models.Sequential([
        layers.Input(shape=(224,224,3)),
        hub.KerasLayer(model_url,
                        trainable=False,
                        name = "feature_extraction_layer"),  # It freezes already learnt pattern and not make any changes,
        layers.Dense(num_of_class,activation=tf.keras.activations.softmax)
    ])
    return model
    

In [17]:
resnet_model=create_model(resnet,10)

ValueError: Exception encountered when calling layer "feature_extraction_layer" (type KerasLayer).

in user code:

    File "/home/ujjwal/miniconda3/envs/tf2.13/lib/python3.9/site-packages/tensorflow_hub/keras_layer.py", line 242, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (4 total):
        * <tf.Tensor 'inputs:0' shape=(None, 224, 224, 3) dtype=float16>
        * False
        * False
        * 0.99
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='inputs')
        * True
        * True
        * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (4 total):
        * Ten

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['mae','accuracy'])

In [ ]:
resnet_history = resnet_model.fit(train_data,
                 epochs=5,
                 steps_per_epoch=len(train_data),
                 validation_data = test_data,
                 validation_steps=len(test_data),
                 callbacks=[create_tensorboard_callback("tensorflow_hub","resnet50-v2")])

In [ ]:
# Plot loss curves...
import matplotlib.pyplot as plt
def plot_loss_curves(history):
    # returns seperate loss curves
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"]))

    #plot loss:
    plt.figure(figsize=(15,8))
    plt.subplot(1,2,1)
    plt.plot(epochs,loss,label="training_loss")
    plt.plot(epochs,val_loss,label="val_loss")
    plt.title("loss")
    plt.xlabel("epochs")
    plt.legend()
    #plt.show()

    #Plot the accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs,accuracy,label="training_accuracy")
    plt.plot(epochs,val_accuracy,label="val_accuracy")
    plt.title("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    #plt.show()

In [ ]:
plot_loss_curves(resnet_history)

#### test out efficientNetB0

In [ ]:
efficient_net = tf.keras.models.Sequential([
    hub.KerasLayer(efficientnet,
                   trainable=False,
                   input_shape=(224,224,3),
                   name="efficientNetB0"),
    tf.keras.layers.Dense(10,activation=tf.keras.activations.softmax)
])
efficient_net.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                      metrics=['accuracy'])
history_efficient = efficient_net.fit(train_data,
                                      epochs=5,
                                      steps_per_epoch=len(train_data),
                                      validation_data = test_data,
                                      validation_steps=len(test_data),
                                      callbacks=[create_tensorboard_callback("tensorflow_hub","efficientNetB0")])

In [ ]:
plot_loss_curves(history_efficient)

In [ ]:
efficient_net.summary()

In [ ]:
resnet_model.summary()

In [ ]:
# Check the number of layers we have in the model
efficient_net.layers

In [ ]:
efficient_net.layers[0].weights,len(efficient_net.layers[0].weights)